In [7]:
#files
import sys
from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
from os.path import isfile, join


#own libs
import utils
import caract as dc
import dataset as ds


#data and visualization
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go


#torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset


sns.set_style("darkgrid")

## **data**

In [8]:
#fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/high/"
fpath =  "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
onlyfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]

cutoff = [8/1000, 11/1000]

#load data
frames = []
for fname in onlyfiles:
    if fname.endswith(".mat"):
        set =  ds.MatFileToDataFrame(fpath, fname)
        df = set.get_dataframe(cutoff)
        frames.append(df)
        print(fname)

frames = frames[:2]

file10_080124.mat
file11_080124.mat
file12_080124.mat
file13_080124.mat
file15_080124.mat
file16_080124.mat
file17_080124.mat
file18_080124.mat
file1_050124.mat
file1_080124.mat
file2_050124.mat
file3_080124.mat
file7_080124.mat
file8_080124.mat
file9_080124.mat
low1_100124.mat
low2_100124.mat
low3_100124.mat
low5_100124.mat


In [9]:
defectos_frames = []
taus = []
for i, df in enumerate(frames):
    aux = dc.CaractDefect(df)
    defectos_frames.append(aux)
    taus.append(aux.get_tau()[1])
    sys.stdout.write(str(i) + '\r')

In [10]:
hilbert_frames = []
for i, df in enumerate(defectos_frames):
    aux, _ = df.get_hilbert()
    hilbert_frames.append(aux)
    sys.stdout.write(str(i) + '\r')

In [11]:
# all frames
frames_all = []

for i in range(len(frames)):
    frames_all.append(pd.merge(frames[i], hilbert_frames[i], on='Hilbert Transform', how='outer'))

frames_all[0]

,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform,Amplitude,Instantaneous Phase,Gradient Phase
0,0.076967,-0.041045,-0.005045,-0.005045+0.003866j,0.006356,2.487745,-0.050221
1,0.076638,-0.041374,-0.004841,-0.004841+0.000279j,0.004849,3.083952,-0.023636
2,0.073022,-0.044991,-0.004633,-0.004633+0.000083j,0.004634,3.123610,0.001438
3,0.073022,-0.044991,-0.004421,-0.004421-0.001203j,0.004582,3.407171,0.023619
4,0.071378,-0.046635,-0.004205,-0.004205-0.001356j,0.004419,3.453607,0.041826
...,...,...,...,...,...,...,...
599995,0.176268,0.058256,0.000015,0.000015+0.000493j,0.000493,30135.697377,-0.289769
599996,0.177254,0.059242,0.000033,0.000033+0.000343j,0.000345,30135.630801,-0.245612
599997,0.179227,0.061215,0.000050,0.000050+0.001284j,0.001285,30135.688260,-0.187389
599998,0.179885,0.061872,0.000066,0.000066+0.001095j,0.001097,30135.667052,-0.118399


### **series for training**

In [12]:
frames_all[0].to_csv('frame0.csv', index=False)

In [13]:
serie = frames_all[0][['Filtered Signal','Amplitude', 'Gradient Phase']].values
serie

array([[-5.04470124e-03,  6.35553595e-03, -5.02207150e-02],
       [-4.84089753e-03,  4.84895059e-03, -2.36363872e-02],
       [-4.63294633e-03,  4.63369556e-03,  1.43767199e-03],
       ...,
       [ 5.04539508e-05,  1.28514434e-03, -1.87389303e-01],
       [ 6.63158304e-05,  1.09719893e-03, -1.18399108e-01],
       [ 8.07604033e-05,  4.17810223e-03, -4.27671146e-02]])

In [27]:
def load_data(Serie,size_train = 0.8, size_val = 0.3, batch_size=32):
    trainval_size = int(size_train*len(serie))
    val_size =  int(size_val*trainval_size)
    train_size = trainval_size - val_size

    X, y = utils.create_sequences(serie, 1000, 100, 0)
    
    #split de data
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:trainval_size], y[train_size:trainval_size]
    X_test, y_test = X[trainval_size:], y[trainval_size:]

    x_train = torch.from_numpy(X_train).float()
    x_val = torch.from_numpy(X_val).float()
    x_test = torch.from_numpy(X_test).float()

    y_train = torch.from_numpy(y_train).float()
    y_val = torch.from_numpy(y_val).float()
    y_test = torch.from_numpy(y_test).float()
    
    #create dataset tensor
    train_dataset = TensorDataset(x_train, y_train)
    val_dataset = TensorDataset(x_val, y_val)
    test_dataset = TensorDataset(x_test, y_test)

    #create dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return train_dataloader, val_dataloader, test_dataloader

In [28]:
train_dataloader, val_dataloader, test_dataloader = load_data(serie)

## **modelo 1**

In [ ]:
class LSTMModel(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        #self.sigmoid = nn.Sigmoid(hidden_size)



    def forward(self, x):
        
        h0 =  torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 =  torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
input_size = 3
hidden_size = 50
num_layers = 1
output_size = 2

model = LSTMModel(input_size, hidden_size, output_size, num_layers)

In [ ]:
# entrenamiento
lr = 0.01
epochs = 100

# loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    
    optimizer.zero_grad()
    
    for X_batch, y_batch in train_dataloader:
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    # validation step
    model.eval()
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_dataloader:
            val_pred = model(X_val_batch)
            val_loss = criterion(val_pred, y_val_batch)
            print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}') 

    model.train()